In [357]:
import customtkinter
from tkinter import filedialog


In [358]:
from typing import Callable, List
import threading

In [359]:
import json

In [360]:
import os

In [361]:
import boto3

In [362]:
extensiones_validas = (
	".pdf", ".doc", ".docx", ".xls", ".xlsx",
	".ppt", ".pptx", ".txt", ".jpg", ".jpeg", ".png", ".gif", ".bmp", ".webp", ".svg"
)

In [363]:
def archivo_es_valido(ruta_archivo):
	return os.path.splitext(ruta_archivo)[1].lower() in extensiones_validas

In [364]:
config = {
	"access_key": "",
	"secret_key": "",
	"bucket": "",
	"region": "us-east-1"
}

In [365]:
CONFIG_FILE = "aws_config.json"
def guardar_config_archivo():
	with open(CONFIG_FILE, "w") as f:
			json.dump(config, f)

def cargar_config_archivo():
	if os.path.exists(CONFIG_FILE):
		try:
			with open(CONFIG_FILE, "r") as f:
					return json.load(f)
		except json.JSONDecodeError:
			print("⚠️ El archivo de configuración está vacío o dañado. Se ignorará.")
			return {}
	return {}

In [366]:
def url_es_de_s3(url: str) -> bool:
	"""
	Retorna True si la URL corresponde a un objeto S3 (formato típico).
	"""
	return url.startswith("https://") and ".s3." in url


In [367]:
def cerrar_app(app: customtkinter):
	app.destroy()  # Cierra la ventana
	app.quit()     # Detiene el mainloop correctamente

In [368]:
global archivo_seleccionado
archivo_seleccionado = ""

In [369]:
def obtener_buckets(access_key: str, secret_key: str, region: str) -> List[str]:
	try:
		s3 = boto3.client(
			"s3",
			aws_access_key_id=access_key,
			aws_secret_access_key=secret_key,
			region_name=region
		)
		response = s3.list_buckets()
		nombres = [bucket["Name"] for bucket in response["Buckets"]]
		return nombres
	except Exception as e:
		print(f"❌ Error al obtener buckets: {e}")
		return []
	
def actualizar_lista_buckets(
	entry_access_key: customtkinter.CTkEntry,
	entry_secret_key: customtkinter.CTkEntry,
	entry_region: customtkinter.CTkEntry,
	menu_bucket: customtkinter.CTkOptionMenu,
	label_confirm: customtkinter.CTkLabel,
) -> None:
	buckets = obtener_buckets(
		entry_access_key.get(),
		entry_secret_key.get(),
		entry_region.get()
	)
	if buckets:
		menu_bucket.configure(values=buckets)
		menu_bucket.set(buckets[0])  # Selecciona el primero por defecto
		label_confirm.configure(text="✅ Buckets cargados")
	else:
		label_confirm.configure(text="❌ No se pudieron cargar buckets")



In [370]:
def limpiar_campos(
				entry_access_key: customtkinter.CTkEntry,
				entry_secret_key: customtkinter.CTkEntry,
				entry_region: customtkinter.CTkEntry,
				menu_bucket: customtkinter.CTkOptionMenu,
				label_confirm: customtkinter.CTkLabel
) -> None:
		entry_access_key.delete(0, "end")
		entry_secret_key.delete(0, "end")
		entry_region.delete(0, "end")
		menu_bucket.set("Sin cargar")
		label_confirm.configure(text="")

In [371]:
def guardar_config(
		entry_access_key: customtkinter.CTkEntry,
		entry_secret_key: customtkinter.CTkEntry,
		entry_region: customtkinter.CTkEntry,
		menu_bucket: customtkinter.CTkOptionMenu,
		label_confirm: customtkinter.CTkLabel
) -> None:
	config["access_key"] = entry_access_key.get()
	config["secret_key"] = entry_secret_key.get()
	config["bucket"] = menu_bucket.get()
	config["region"] = entry_region.get()
	guardar_config_archivo()
	label_confirm.configure(text="✅ Configuración guardada con éxito")

In [372]:
def seleccionar_archivo(
		label_archivo: customtkinter.CTkLabel,
		textbox_name_file: customtkinter.CTkEntry,
		textbox_url: customtkinter.CTkTextbox,
		boton_subir: customtkinter.CTkButton,
		boton_hacer_publico: customtkinter.CTkButton,
		boton_copiar: customtkinter.CTkButton
):
	global archivo_seleccionado
	archivo = filedialog.askopenfilename()
	if archivo_es_valido(archivo):
		archivo_seleccionado = archivo
		label_archivo.configure(text=f"Archivo seleccionado:\n{archivo}")

		# Establece por defecto el nombre original en el textbox (sin la ruta)
		nombre_original = os.path.basename(archivo)
		textbox_name_file.delete(0, "end")
		textbox_name_file.insert(0, nombre_original)

		textbox_url.configure(state="normal")
		textbox_url.delete("0.0", "end")
		textbox_url.configure(state="disabled")

		boton_subir.configure(state="normal")
		boton_hacer_publico.configure(state="disabled")
		boton_copiar.configure(state="disabled")
	else:
		archivo_seleccionado = ""
		label_archivo.configure(text="El archivo seleccionado no es válido.")

In [383]:
def crear_loader_padre(parent: customtkinter.CTkBaseClass, *, width: int = 300, pady: int = 5) -> customtkinter.CTkProgressBar:
    """
    Crea una CTkProgressBar 'indeterminate', la deja oculta y
    devuelve la referencia.
    El contenedor `parent` DEBE usar el gestor de geometría pack().
    """
    loader = customtkinter.CTkProgressBar(parent, mode="indeterminate", width=width)
    loader.pack(pady=pady)    # la empaquetamos…
    loader.stop()             # …pero sin animarla todavía
    loader.pack_forget()      # …y la ocultamos
    return loader

def mostrar_loader(loader: customtkinter.CTkProgressBar) -> None:
	# vuelve a “empacar” la barra (por si estaba oculta)
	loader.pack(pady=5)
	loader.start()

def ocultar_loader(loader: customtkinter.CTkProgressBar) -> None:
	loader.stop()
	loader.pack_forget()

def crear_loader_grid(parent, row=0, column=0, columnspan=1, width=300, pady=5):
    loader = customtkinter.CTkProgressBar(parent, mode="indeterminate", width=width)
    loader.grid(row=row, column=column, columnspan=columnspan, pady=pady)
    loader.stop()
    loader.grid_remove()
    return loader

def mostrar_loader_grid(loader):  loader.grid(); loader.start()
def ocultar_loader_grid(loader):  loader.stop(); loader.grid_remove()

In [374]:
def hacer_publico_ultimo_archivo(
		textbox_url: customtkinter.CTkTextbox,
		label_archivo: customtkinter.CTkLabel,
		entry_access_key: customtkinter.CTkEntry,
		entry_secret_key: customtkinter.CTkEntry,
		entry_region: customtkinter.CTkEntry,
		menu_bucket: customtkinter.CTkOptionMenu,
		boton_hacer_publico: customtkinter.CTkButton
):
	url = textbox_url.get("0.0", "end").strip()
	# ⛔ 1. Si no hay URL, nada que hacer
	if not url:
			label_archivo.configure(text="⚠️ Primero sube un archivo.")
			return
	if not url_es_de_s3(url):
			label_archivo.configure(text="⚠️ La URL no parece ser de Amazon S3.")
			return
	try:
		s3 = boto3.client(
			"s3",
			aws_access_key_id=entry_access_key.get(),
			aws_secret_access_key=entry_secret_key.get(),
			region_name=entry_region.get()
		)

		nombre_bucket = menu_bucket.get()
		nombre_objeto = url.split("/")[-1]

		s3.put_object_acl(Bucket=nombre_bucket, Key=nombre_objeto, ACL='public-read')

		label_archivo.configure(text=f"🌍 Archivo ahora es público:\n{nombre_objeto}")
		# # ✔️ Deshabilitamos el botón; ya quedó público
		boton_hacer_publico.configure(state="disabled")
	except Exception as e:
		label_archivo.configure(text=f"❌ Error al cambiar visibilidad:\n{e}")

In [375]:
def copiar_url(
		root: customtkinter.CTk,
		textbox_url: customtkinter.CTkTextbox,
		label_archivo: customtkinter.CTkLabel,
) -> None:
	url = textbox_url.get("0.0", "end").strip()
	if not url:
		label_archivo.configure(text="⚠️ No hay URL para copiar")
		return
	root.clipboard_clear()
	root.clipboard_append(url)
	label_archivo.configure(text="📋 URL copiada al portapapeles")

In [376]:
def subir_archivo_a_s3(
		entry_access_key: customtkinter.CTkEntry,
		entry_secret_key: customtkinter.CTkEntry,
		entry_region: customtkinter.CTkEntry,
		menu_bucket: customtkinter.CTkOptionMenu,
		textbox_name_file: customtkinter.CTkEntry,
		es_publico: customtkinter.CTkCheckBox,
		label_archivo: customtkinter.CTkLabel,
		textbox_url: customtkinter.CTkTextbox,
		boton_subir: customtkinter.CTkButton,
		boton_copiar: customtkinter.CTkButton,
		boton_hacer_publico: customtkinter.CTkButton
):
	global archivo_seleccionado
	if not archivo_seleccionado:
		return

	try:
		s3 = boto3.client(
			"s3",
			aws_access_key_id=entry_access_key.get(),
			aws_secret_access_key=entry_secret_key.get(),
			region_name=entry_region.get()
		)

		nombre_bucket = menu_bucket.get()
		nombre_personalizado = textbox_name_file.get().strip()
		nombre_objeto = nombre_personalizado if nombre_personalizado else os.path.basename(archivo_seleccionado)

		import re
		nombre_objeto = re.sub(r"[^\w\-. ]", "_", nombre_objeto)

		if "." not in nombre_objeto:
				extension = os.path.splitext(archivo_seleccionado)[1]
				nombre_objeto += extension

		extra_args = {}
		if es_publico.get():
				extra_args['ACL'] = 'public-read'

		with open(archivo_seleccionado, "rb") as f:
			if extra_args:
				s3.upload_fileobj(f, nombre_bucket, nombre_objeto, ExtraArgs=extra_args)
			else:
				s3.upload_fileobj(f, nombre_bucket, nombre_objeto)

		# URL pública
		region = entry_region.get()
		url = f"https://{nombre_bucket}.s3.{region}.amazonaws.com/{nombre_objeto}"

		label_archivo.configure(text=f"✅ Archivo subido:\n{nombre_objeto}")
		textbox_url.configure(state="normal")
		textbox_url.delete("0.0", "end")
		textbox_url.insert("0.0", url)
		textbox_url.configure(state="disabled")
		boton_subir.configure(state="disabled")
		boton_copiar.configure(state="normal")
		if not es_publico.get():
			boton_hacer_publico.configure(state="normal")
	except Exception as e:
			label_archivo.configure(text=f"❌ Error al subir archivo:\n{e}")

In [377]:
def subir_archivo_worker(*args, loader: customtkinter.CTkProgressBar, root: customtkinter.CTk, **kwargs):
    """
    Corre subir_archivo_a_s3 en un hilo y oculta el loader al terminar,
    sin importar si hubo error o no.
    """
    try:
        subir_archivo_a_s3(*args, **kwargs)
    finally:
        # volvemos al hilo principal para tocar la GUI
        root.after(0, lambda: ocultar_loader(loader))


In [382]:
def actualizar_lista_worker(*args, loader: customtkinter.CTkProgressBar, root: customtkinter.CTk, **kwargs):
    try:
        actualizar_lista_buckets(*args, **kwargs)
    finally:
        root.after(0, lambda: ocultar_loader(loader))
        

In [379]:
from typing import Any, Dict

def crear_tab_subir(root: customtkinter.CTk, tab: customtkinter.CTkFrame, refs: Dict[str, Any]) -> Dict[str, Any]:
		# --- Widgets principales ---
		label_archivo = customtkinter.CTkLabel(tab, text="Ningún archivo seleccionado aún.")
		label_archivo.pack(pady=10)

		# Nombre de archivo
		frame_nombre = customtkinter.CTkFrame(tab)
		frame_nombre.pack(pady=5)

		customtkinter.CTkLabel(frame_nombre, text="Nombre del Archivo").grid(row=0, column=0, padx=5, pady=5, sticky="e")
		textbox_name = customtkinter.CTkEntry(frame_nombre, placeholder_text="Nombre del Archivo", width=300)
		textbox_name.grid(row=0, column=1, padx=5, pady=5, sticky="w")

		# URL
		customtkinter.CTkLabel(tab, text="URL del archivo en S3").pack()
		textbox_url = customtkinter.CTkTextbox(tab, height=40, width=500, state="disabled")
		textbox_url.pack(pady=5)

		# Botones
		boton_copiar = customtkinter.CTkButton(tab, text="Copiar URL", state="disabled")
		boton_copiar.pack(pady=5)

		es_publico = customtkinter.CTkCheckBox(tab, text="¿Hacer público?")
		es_publico.select()
		es_publico.pack(pady=5)

		boton_seleccionar = customtkinter.CTkButton(tab, text="Seleccionar archivo", state="normal")
		boton_subir       = customtkinter.CTkButton(tab, text="Subir archivo", state="disabled")
		boton_publico     = customtkinter.CTkButton(tab, text="Hacer público", state="disabled")

		boton_seleccionar.pack(pady=10)
		boton_subir.pack(pady=10)
		boton_publico.pack(pady=5)

		# ---- Conectar callbacks (lambda o funciones aparte) ----
		boton_seleccionar.configure(
				command=lambda: seleccionar_archivo(
						label_archivo, textbox_name, textbox_url,
						boton_subir, boton_publico, boton_copiar
				)
		)

		loader = crear_loader_padre(tab)

		def lanzar_subida():
			mostrar_loader(loader)
			threading.Thread(
				target=subir_archivo_worker,
				kwargs=dict(
					entry_access_key = refs["entry_access"],
            		entry_secret_key = refs["entry_secret"],
            		entry_region     = refs["entry_region"],
            		menu_bucket      = refs["menu_bucket"],
            		textbox_name_file= textbox_name,
            		es_publico       = es_publico,
            		label_archivo    = label_archivo,
            		textbox_url      = textbox_url,
            		boton_subir      = boton_subir,
            		boton_copiar     = boton_copiar,
            		boton_hacer_publico = boton_publico,
            		loader           = loader,
            		root             = root
				),
				daemon=True
			).start()

		boton_subir.configure(command=lanzar_subida)
		
		boton_copiar.configure(
				command=lambda: copiar_url(root, textbox_url, label_archivo)
		)

		boton_publico.configure(
				command=lambda: hacer_publico_ultimo_archivo(
						textbox_url, label_archivo,
						refs["entry_access"], refs["entry_secret"], refs["entry_region"],
						refs["menu_bucket"], boton_publico
				)
		)

		# Devolver referencias compartidas
		return {
				"label_archivo":   label_archivo,
				"textbox_name":    textbox_name,
				"textbox_url":     textbox_url,
				"boton_subir":     boton_subir,
				"boton_publico":   boton_publico,
				"boton_copiar":    boton_copiar,
				"es_publico":      es_publico,
		}


In [380]:
def crear_tab_config(tab: customtkinter.CTkFrame, refs: Dict[str, Any]) -> None:
		tab.grid_columnconfigure(0, weight=1)
		tab.grid_columnconfigure(1, weight=1)
		customtkinter.CTkLabel(tab, text="🔐 Configuración AWS").grid(row=0, column=0, columnspan=2, pady=(0, 10))

		# Entradas
		entry_access = customtkinter.CTkEntry(tab, placeholder_text="Access Key ID");   entry_access.grid(row=1, column=1, padx=5, pady=5)
		entry_secret = customtkinter.CTkEntry(tab, placeholder_text="Secret Access Key", show="*"); entry_secret.grid(row=2, column=1, padx=5, pady=5)
		entry_region = customtkinter.CTkEntry(tab, placeholder_text="us-east-1");        entry_region.grid(row=3, column=1, padx=5, pady=5)

		# Labels
		customtkinter.CTkLabel(tab, text="Access Key ID").grid(row=1, column=0, sticky="e", padx=5, pady=5)
		customtkinter.CTkLabel(tab, text="Secret Access Key").grid(row=2, column=0, sticky="e", padx=5, pady=5)
		customtkinter.CTkLabel(tab, text="Región").grid(row=3, column=0, sticky="e", padx=5, pady=5)

		# Menú Bucket y Botones
		menu_bucket = customtkinter.CTkOptionMenu(tab, values=["Sin cargar"]);
		menu_bucket.grid(row=5, column=0, columnspan=2, pady=5)
		
		datos = cargar_config_archivo()
		if datos:
			entry_access.insert(0, datos.get("access_key", ""))
			entry_secret.insert(0, datos.get("secret_key", ""))
			entry_region.insert(0, datos.get("region", ""))
			menu_bucket.set(datos.get("bucket", "Sin cargar"))
			config.update(datos)
		
		boton_cargar = customtkinter.CTkButton(tab, text="Cargar Buckets", command=lambda: actualizar_lista_buckets(
				entry_access, entry_secret, entry_region, menu_bucket, label_confirm
		))
		boton_cargar.grid(row=4, column=0, columnspan=2, pady=5)

		label_confirm = customtkinter.CTkLabel(tab, text="");
		label_confirm.grid(row=9, column=0, columnspan=2, pady=5)

		# Guardar y limpiar
		boton_guardar = customtkinter.CTkButton(tab, text="Guardar configuración", command=lambda: guardar_config(
				entry_access, entry_secret, entry_region, menu_bucket, label_confirm
		));
		boton_guardar.grid(row=7, column=0, columnspan=2, pady=20)

		boton_limpiar = customtkinter.CTkButton(tab, text="Limpiar campos", command=lambda: limpiar_campos(
				entry_access, entry_secret, entry_region, menu_bucket, label_confirm
		));
		boton_limpiar.grid(row=8, column=0, columnspan=2, pady=(5, 20))

		# 👉 Almacena referencias para la pestaña de subir
		refs.update(
				entry_access=entry_access,
				entry_secret=entry_secret,
				entry_region=entry_region,
				menu_bucket=menu_bucket,
				label_confirm=label_confirm
		)


In [381]:
def mostrar_ventana() -> None:
	app = customtkinter.CTk()
	app.geometry("600x550")
	app.resizable(False, False)
	app.title("Subir Archivo a AWS")

	# Encabezado
	customtkinter.CTkLabel(app, text="Subir Archivo a Amazon S3").pack(pady=10)

	# Pestañas
	tabview = customtkinter.CTkTabview(app, width=690, height=550)
	tabview.pack(padx=10, pady=10)

	# --- Pestaña SUBIR ---
	tab_subir        = tabview.add("Subir archivo")
	tab_conf       = tabview.add("Configuración")

	# Construir cada pestaña con funciones auxiliares
	refs: Dict[str, Any] = {}                 # diccionario vacío
	crear_tab_config(tab_conf, refs)
	refs |= crear_tab_subir(app, tab_subir, refs)   # llena refs

	# Handler de cierre
	app.protocol("WM_DELETE_WINDOW", lambda: cerrar_app(app))
	app.mainloop()
	
mostrar_ventana()